## Pre-Processing Interfaces

#### Values Management Policy
- Lines with duplicate smiles will be dropped in favor of the one with lowest ```Energy_```, unless another
column is provided.
- Lines with ```nan```/missing values are dropped. 

In [63]:
import numpy as np
import pandas as pd
import IPython

def ldf_display(df, nlines=500):
    """Utility function to display large dataframe portions as an html table."""
    txt = ("<iframe " +
           "srcdoc='" + df.head(nlines).to_html() + "' " +
           "width=1000 height=500 style=\"background: #FFFFFF;\">" +
           "</iframe>")
    return IPython.display.HTML(txt)

def prune_dataset_lines(
    dataset: pd.DataFrame, remove_nan: bool = True, 
    remove_duplicates_: bool = True, in_favour_of_col: str = 'Energy_(kcal/mol)') -> pd.DataFrame:
    """
    Remove lines from dataset if they contain nan values or are missing values.
    This function does not modify it's parameters.

    Parameters
    -----------
    - dataset the dataset to prune. It is copied inside the function. 
    - remove_nan
    - remove_duplicates
    - in_favour_of_col
    
    Returns
    -----------
    The pruned DataFrame by copy. 
    """
    pruned_dataset = dataset.copy()
    pruned_dataset.replace("nan", np.nan)
    
    if remove_nan:  
        pruned_dataset.dropna(axis=0, how='any', inplace=True)
    
    if remove_duplicates_:
        pruned_dataset.sort_values(axis=0, by=['Chiral_Molecular_SMILES', in_favour_of_col], ascending=True)
        pruned_dataset.drop_duplicates(subset='Chiral_Molecular_SMILES', keep='first', inplace=True)
    
    return pruned_dataset

## Smiles Encoding Interface

#### Supported strategies
- One hot encoding (problem : explodes dimensionality)
- Samuel's solutions
- ToDo : Number solution (convert the binary to base10)

In [64]:
def encode_smiles_column_of(dataset: pd.DataFrame, strategy: str = 'one_hot_encoding'):
    """
    Encode smiles string column to numerical representation. 
    This function does not modify it's arguments.

    Parameters
    -----------
    - dataset the dataset to prune. It is copied inside the function. 
    - strategy one of 'one_hot_encoding', 'count_encoding'
    
    Returns
    -----------
    The encoded dataset by copy.
    """
    encoded_dataset = dataset.copy()
    smiles = encoded_dataset['Chiral_Molecular_SMILES']

    if strategy == 'one_hot_encoding':
        encoded_dataset = pd.get_dummies(dataset, columns=['Chiral_Molecular_SMILES'])

    if strategy == 'count_encoding':
        smiles_alphabet = list(set(''.join(np.asanyarray(smiles))))
        for letter in smiles_alphabet:
            encoded_dataset[letter] = encoded_dataset.apply(lambda row: row['Chiral_Molecular_SMILES'].count(letter), axis=1)
        
        del encoded_dataset['Chiral_Molecular_SMILES']

    return encoded_dataset

## Baseline

The chosen baseline algorithm is Linear Regression. 

In [65]:
from sklearn.linear_model import LinearRegression
from utils import load_data
from sklearn.metrics import mean_squared_error
import numpy as np

def normalize_ndarray(X: np.ndarray):
    """Normalize every column of X by substracting mean and dividing by std"""
    for j in range(X.shape[1]):
        if np.std(X[:, j]) != 0: 
            X[:, j] = (X[:, j]-np.mean(X[:, j]))/np.std(X[:, j])
        else:
            X[:, j] = 0
    return X

def return_required_data(dataset: pd.DataFrame, targets: list[str], normalize=True): 
    target_indices = []
    for target in targets:
        target_indices.append(dataset.columns.get_loc(target))

    X_train, X_test = load_data(np.asanyarray(dataset))
    
    y_test = np.ndarray(shape=(X_test.shape[0], len(targets)))
    y_train = np.ndarray(shape=(X_train.shape[0], len(targets)))

    # remove target columns from X and add them to y
    # with train and test data
    for i, index in enumerate(target_indices):                
        y_train[:, i] = X_train[:, index]
        np.delete(X_train, index, 1)

        y_test[:, i] = X_test[:, index]
        np.delete(X_test, index, 1)
    
    if normalize:
        normalize_ndarray(X_train)
        normalize_ndarray(y_train)
        normalize_ndarray(X_test)
        normalize_ndarray(y_test)

    return X_train, y_train, X_test, y_test

In [68]:
raw_dataset = pd.read_csv('./dataset/data.csv')

pruned_dataset = prune_dataset_lines(raw_dataset)
print("Raw dataset shape =", raw_dataset.shape, " Pruned dataset shape =", pruned_dataset.shape)

encoded_pruned_data = encode_smiles_column_of(pruned_dataset, 'count_encoding') # change to count_encoding here
print("Encoded dataset shape =", encoded_pruned_data.shape)

X_train, y_train, X_test, y_test = return_required_data(encoded_pruned_data, ['Energy_(kcal/mol)', 'Energy DG:kcal/mol)'], normalize=True)

LR = LinearRegression()
LR = LR.fit(X_train, y_train)

y_pred_test = LR.predict(X_test)
y_pred_train = LR.predict(X_train)

print(mean_squared_error(y_test, y_pred_test))
print(mean_squared_error(y_train, y_pred_train))
print(np.abs(y_pred_test-y_test)/y_test)


Raw dataset shape = (39926, 24)  Pruned dataset shape = (3287, 24)
Encoded dataset shape = (3287, 52)
1.7818511968171326e-20
2.0666718876780286e-30
[[-1.29407065e-10 -6.94946087e-11]
 [-4.55979976e-10 -4.19873423e-10]
 [-2.58550673e-11 -1.10674702e-10]
 ...
 [-2.03265096e-11 -6.00283107e-11]
 [-1.95709734e-10 -1.15286832e-10]
 [-8.64724341e-11 -1.92563967e-10]]


/home/gordon/.local/lib/python3.10/site-packages/numpy/core/_methods.py:235: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
